In [ ]:
# default_exp utils

# fcst_campaign utils module

> Some generic tools

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip install git+https://github.com/pete88b/nbdev_colab_helper.git
  from nbdev_colab_helper.core import *
  project_name = 'forecast_action'
  init_notebook(project_name)

  Cloning https://github.com/pete88b/nbdev_colab_helper.git to /tmp/pip-req-build-38eit630
  Running command git clone --filter=blob:none --quiet https://github.com/pete88b/nbdev_colab_helper.git /tmp/pip-req-build-38eit630
  Resolved https://github.com/pete88b/nbdev_colab_helper.git to commit 15cf67b6ca0c125a636cedc017f285a04771bbff
  Preparing metadata (setup.py) ... done
  Created wheel for nbdev-colab-helper: filename=nbdev_colab_helper-0.0.1-py3-none-any.whl size=9679 sha256=83f342c034baedf828bf009aba857a57ff470f507bbbe8a297e1b91638d23ad0
  Stored in directory: /tmp/pip-ephem-wheel-cache-srf4vk1b/wheels/0c/50/90/0c073bb88ab36268043083b5ee7bc3a8b39c1a423c3c3736b7
Successfully built nbdev-colab-helper
Connecting to google drive
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/github/forecast_action
pip install fastscript==1.0.0 fastcore==1.0.8 nbdev==1.0.14
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#hide
from nbdev.showdoc import *

if IN_COLAB:
  !pip install timezonefinder



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.6 MB/s eta 0:00:00
  Created wheel for timezonefinder: filename=timezonefinder-6.4.1-cp310-cp310-manylinux_2_35_x86_64.whl size=49040891 sha256=a43f7e7ad38098d576217177bc26b0740cdb55f26a1d8cb5b292529192fa4fa1
  Stored in directory: /root/.cache/pip/wheels/52/8f/5f/8ace889092b27a1c13b9fa7582d5763f417c29b9d23388415d
Successfully built timezonefinder


In [ ]:
# export
import pytz
from datetime import datetime
from timezonefinder import TimezoneFinder

In [ ]:
# export
def get_state_capitals(geo_entry):
    # Maps of country to a list of its state capitals
    capitals = {
        "USA": ["Washington, D.C.", "Austin, TX", "Sacramento, CA", "New York, NY", "Chicago, IL", "Denver, CO", "Kanas City, MO"],
        "India": ["New Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad", "Pune"]
    }

    # Check if the geo_entry is a recognized country and return its capitals
    if geo_entry in capitals:
        return capitals[geo_entry]
    else:
        # Return an empty list or a meaningful message if no data is available
        return []

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [ ]:
# export
def get_lat_lon(geo_input):
    """
    Convert a geographical input (city, state, or country) into a list of latitude-longitude pairs.
    """
    geolocator = Nominatim(user_agent="forecastcampaignad/vjsoc77@gmail.com",timeout=10)
    # Use RateLimiter to avoid hitting usage limits
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    lat_lon_list = []

    # Example of handling different types of geographical inputs
    if isinstance(geo_input, str):  # Single city, state, or country
        location = geolocator.geocode(geo_input)
        if location:
            lat_lon_list.append((location.latitude, location.longitude))
    elif isinstance(geo_input, list):  # List of cities
        for item in geo_input:
            location = geolocator.geocode(item)
            if location:
                lat_lon_list.append((location.latitude, location.longitude))
            time.sleep(1)

    return lat_lon_list

In [ ]:
# export
def create_geo_query_list(company_df, company_name):
    """
    For a given company, generate a list of latitude-longitude pairs based on its 'Geo' field.
    """
    # Find the 'Geo' entry for the company
    geo_entry = company_df[company_df['Company'] == company_name]['Geo'].iloc[0]
    print(geo_entry)

    country_names = ['USA', 'India']
    ## Get a list of cities
    if geo_entry in country_names:
      city_names = get_state_capitals(geo_entry)
    else:
      city_names = [geo_entry]

    # Handling different Geo formats (e.g., city, state, country)
    # This is a placeholder; implementation depends on your data and needs
    lat_lon_list = get_lat_lon(city_names)

    return lat_lon_list, city_names

In [ ]:
# export
def round_to_grid(lat, lon, resolution=0.25):
    """Round latitude and longitude to the nearest grid resolution."""
    rounded_lat = round(lat / resolution) * resolution
    rounded_lon = round(lon / resolution) * resolution
    return rounded_lat, rounded_lon

# Example: Rounding the coordinates for Chicago
chicago_lat, chicago_lon = 41.8781, -87.6298
rounded_lat, rounded_lon = round_to_grid(chicago_lat, chicago_lon)

print(f"Rounded Coordinates for Chicago: Latitude {rounded_lat}, Longitude {rounded_lon}")

Rounded Coordinates for Chicago: Latitude 42.0, Longitude -87.75


In [ ]:
# export
def get_local_peak_hours(lat_lons):
    tf = TimezoneFinder()
    peak_hours_info = []

    for lat, lon in lat_lons:
        # Find the time zone for the given lat-lon
        tz_str = tf.timezone_at(lat=lat, lng=lon)
        tz = pytz.timezone(tz_str)

        # Define peak hours in UTC
        utc_peak_start = datetime.utcnow().replace(hour=10, minute=0, second=0, microsecond=0)
        utc_peak_end = datetime.utcnow().replace(hour=17, minute=0, second=0, microsecond=0)

        # Convert peak hours to local time
        local_peak_start = utc_peak_start.astimezone(tz)
        local_peak_end = utc_peak_end.astimezone(tz)

        peak_hours_info.append({
            "lat_lon": (lat, lon),
            "time_zone": tz_str,
            "local_peak_start": local_peak_start.strftime("%Y-%m-%d %H:%M:%S"),
            "local_peak_end": local_peak_end.strftime("%Y-%m-%d %H:%M:%S"),
        })

    return peak_hours_info

# Example usage
lat_lons = [(40.7128, -74.0060), (28.6139, 77.2090)]  # New York, New Delhi
peak_hours = get_local_peak_hours(lat_lons)
for info in peak_hours:
    print(info)


{'lat_lon': (40.7128, -74.006), 'time_zone': 'America/New_York', 'local_peak_start': '2024-03-07 05:00:00', 'local_peak_end': '2024-03-07 12:00:00'}
{'lat_lon': (28.6139, 77.209), 'time_zone': 'Asia/Kolkata', 'local_peak_start': '2024-03-07 15:30:00', 'local_peak_end': '2024-03-07 22:30:00'}


In [ ]:
# export
def get_timezone_offset(lat, lon, date=datetime.now()):
    """
    Returns the timezone offset in hours for a given latitude, longitude, and date.

    :param lat: Latitude of the location
    :param lon: Longitude of the location
    :param date: Date for which to find the timezone offset, default is current date
    :return: Timezone offset in hours from UTC
    """
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=lat, lng=lon)  # Find the timezone name
    if timezone_str is None:
        return None  # Could not determine the timezone
    timezone = pytz.timezone(timezone_str)

    # Calculate offset in hours, considering daylight saving time
    offset = timezone.utcoffset(date)
    offset_hours = offset.total_seconds() / 3600
    return offset_hours

# Example usage
lat, lon = 40.7128, -74.0060  # New York City
offset_hours = get_timezone_offset(lat, lon)
print(f"Timezone offset for latitude {lat}, longitude {lon} is {offset_hours} hours.")

Timezone offset for latitude 40.7128, longitude -74.006 is -5.0 hours.


In [ ]:
# export
def calculate_search_radius(city_lat):
    # Constants
    km_per_degree_latitude = 111  # Approximate km covered by one degree of latitude
    longitude_difference = 0.25  # Grid cell size in degrees (longitude)

    # Calculating the East-West length of the grid cell at the given latitude
    E_W_length_km = km_per_degree_latitude * math.cos(math.radians(city_lat)) * longitude_difference

    # Calculating the radius as half the E-W length
    radius_km = E_W_length_km / 2
    return radius_km

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted index.ipynb.
